In [3]:
import pandas as pd
import OpenDartReader

api_key = '4fb4acc1d37b1dae74040e64a9674e018ccfd7fe'

dart = OpenDartReader(api_key)

In [3]:
df = pd.read_csv('2019_info.csv',dtype = {'corp_code':str})
df.head()

,Unnamed: 0,corp_code,재무제표,부채총계,법인세차감전 순이익,영업이익
0,0,00106641,"55,344,798,000,000","26,366,660,000,000","2,531,104,000,000","2,009,680,000,000"
1,1,00138279,"16,333,401,000,000","9,836,775,000,000","110,351,000,000","420,077,000,000"
2,2,00155355,"1,484,556,453,749","1,021,454,494,037","6,886,489,346","30,570,664,872"
3,3,00190321,"34,061,292,000,000","18,874,339,000,000","979,676,000,000","1,151,065,000,000"
4,4,00181712,"132,020,676,000,000","79,838,041,000,000","2,776,568,000,000","3,949,864,000,000"


In [44]:
d = dart.report(df['corp_code'][2],'직원',2019)
d

,rcept_no,corp_cls,corp_code,corp_name,sexdstn,fo_bbm,reform_bfe_emp_co_rgllbr,reform_bfe_emp_co_cnttk,reform_bfe_emp_co_etc,rgllbr_co,rgllbr_abacpt_labrr_co,cnttk_co,cnttk_abacpt_labrr_co,sm,avrg_cnwk_sdytrn,fyer_salary_totamt,jan_salary_am,rm
0,20200330004464,Y,00155355,풀무원,남,지주사,-,-,-,238,-,3,-,241,7.3,"14,483,694,000","60,098,000",-
1,20200330004464,Y,00155355,풀무원,여,지주사,-,-,-,174,-,26,-,200,5.2,"8,488,317,000","42,442,000",-


In [45]:
columns = ['corp_code','비정규직 수']
employee = pd.DataFrame(columns = columns)


for code in df['corp_code']:
    try:
        data = dart.report(code,'직원',2019)
        if data['rcept_no'][0][:4]=='2020':
            d = data['cnttk_co']
            numeric_values = pd.to_numeric(d, errors='coerce') 
            total = numeric_values.sum()
            employee.loc[len(employee)] = pd.Series({'corp_code':code,'비정규직 수':total})
        else:
            employee.loc[len(employee)] = pd.Series({'corp_code':code,'비정규직 수':0})
    except:
        employee.loc[len(employee)] = pd.Series({'corp_code':code,'비정규직 수':0})
    print(len(employee))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [46]:
employee[employee['비정규직 수']==0]

,corp_code,비정규직 수
0,00106641,0
12,00160047,0
14,00116824,0.0
24,00117577,0.0
40,00120562,0.0
...,...,...
779,00132868,0
780,00134176,0.0
782,00480950,0.0
784,00110750,0


In [47]:
new_df = employee[employee['비정규직 수']==0]
second_try = []
for code in new_df['corp_code']:
    try:
        data = dart.report(code,'직원',2019)
        if data['rcept_no'][0][:4]=='2020':
            d = data['cnttk_co']
            d = d.str.replace(',', '').astype(int)
            total = d.sum()
            second_try.append(total)
        else:
            second_try.append(0)
    except:
        second_try.append(0)
new_df['비정규직 수'] = second_try


C:\Users\pmsda\AppData\Local\Temp\ipykernel_11920\244840909.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['비정규직 수'] = second_try


In [49]:
employee.loc[new_df.index] = new_df
len(employee[employee['비정규직 수']==0])

60

In [50]:
df = pd.read_csv('2019_info.csv',dtype = {'corp_code':str})
new = df.merge(employee,on = 'corp_code',how = 'outer')
new.to_csv('2019_info.csv',index=False)


In [10]:
employees_ = pd.read_csv('비정규직.csv',dtype = {'corp_code':str})
employees_.columns = ['corp_code','비정규직 수','연도']
merged_df = merged_df.merge(employees_,on = ['corp_code','연도'],how = 'outer')
merged_df.head()

,회사명,corp_code,stock_code,연도,ESG,재무제표,부채총계,영업이익,법인세차감전 순이익,intangible,ratio,pre_ratio,out_exec,share_holder,정규직 수,비정규직 수
0,기아,00106641,000270,2022,B+,"73,710,965,000,000","34,367,884,000,000","7,233,120,000,000","7,501,993,000,000",2.905980e+06,25.9,25.3,55.555556,35.62,34260.0,1587.0
1,S-Oil,00138279,010950,2022,A,"19,613,135,000,000","11,129,666,000,000","3,405,166,000,000","2,898,542,000,000",1.064180e+05,30.4,32.1,8.196721,63.44,951.0,51.0
2,풀무원,00155355,017810,2022,A,"2,049,217,755,236","1,502,570,984,984","26,331,500,743","-18,413,093,758",9.776635e+10,-10.1,NaN,29.629630,54.30,470.0,37.0
3,케이티,00190321,030200,2022,A,"40,980,681,000,000","22,565,958,000,000","1,690,088,000,000","1,894,067,000,000",3.129833e+06,39.8,33.2,40.000000,12.68,19823.0,721.0
4,SK,00181712,034730,2022,A+,"194,505,860,000,000","122,697,070,000,000","8,004,710,000,000","5,908,697,000,000",1.270902e+07,25.5,22.8,4.807692,26.69,4422.0,164.0


In [11]:
merged_df.to_csv('ESG_final.csv',index = False)

In [3]:
df = pd.read_csv('2019_info.csv',dtype = {'corp_code':str})
df.head()

,Unnamed: 0,corp_code,재무제표,부채총계,법인세차감전 순이익,영업이익,정직원 수,비정규직 수
0,0,00106641,"55,344,798,000,000","26,366,660,000,000","2,531,104,000,000","2,009,680,000,000",0.0,0.0
1,1,00138279,"16,333,401,000,000","9,836,775,000,000","110,351,000,000","420,077,000,000",726.0,32.0
2,2,00155355,"1,484,556,453,749","1,021,454,494,037","6,886,489,346","30,570,664,872",412.0,29.0
3,3,00190321,"34,061,292,000,000","18,874,339,000,000","979,676,000,000","1,151,065,000,000",22810.0,562.0
4,4,00181712,"132,020,676,000,000","79,838,041,000,000","2,776,568,000,000","3,949,864,000,000",824.0,130.0


In [6]:
df[df['정직원 수']==0]

,Unnamed: 0,corp_code,재무제표,부채총계,법인세차감전 순이익,영업이익,정직원 수,비정규직 수
0,0,00106641,"55,344,798,000,000","26,366,660,000,000","2,531,104,000,000","2,009,680,000,000",0.0,0.0
12,12,00160047,"3,607,086,237,555","212,953,070,700","196,706,021,587","170,875,981,774",0.0,0.0
43,43,00164742,"194,512,220,000,000","118,146,466,000,000","4,163,766,000,000","1,580,164,000,000",0.0,0.0
45,45,00105952,"10,763,512,000,000","6,492,184,000,000","160,438,000,000","351,981,000,000",0.0,0.0
48,48,00140177,"7,054,847,638,519","4,504,006,759,724","199,839,393,215","238,835,955,920",0.0,0.0
82,82,00219097,"1,657,381,460,248","129,126,735,585","26,690,490,888","29,385,396,571",0.0,0.0
83,83,00126308,"4,565,711,799,514","3,270,525,561,644","437,779,765,527","422,717,496,667",0.0,0.0
94,94,00160232,"957,238,168,720","676,238,454,283","23,713,572,228","52,771,937,110",0.0,0.0
95,95,00124540,"9,697,696,817,049","7,209,447,915,772","307,274,974,856","364,061,074,652",0.0,0.0
100,100,00412597,"2,503,844,833,978","774,839,980,801","125,740,946,152","130,344,227,510",0.0,0.0


In [11]:
dart.report('01237540','직원',2019)

,rcept_no,corp_cls,corp_code,corp_name,sexdstn,fo_bbm,reform_bfe_emp_co_rgllbr,reform_bfe_emp_co_cnttk,reform_bfe_emp_co_etc,rgllbr_co,rgllbr_abacpt_labrr_co,cnttk_co,cnttk_abacpt_labrr_co,sm,avrg_cnwk_sdytrn,fyer_salary_totamt,jan_salary_am,rm
0,20200330002884,K,01237540,이녹스첨단소재,남,IT소재,-,-,-,626,-,-,-,626,2.06,"31,427,000,000","50,000,000",-
1,20200330002884,K,01237540,이녹스첨단소재,여,IT소재,-,-,-,40,-,-,-,40,1.97,"1,382,000,000","35,000,000",-


In [14]:
miss = df[df['정직원 수']==df['비정규직 수']]
second_try = []
for code in miss['corp_code']:
    try:
        data = dart.report(code,'직원',2019)
        if data['rcept_no'][0][:4]=='2020':
            d = data['cnttk_co']
            d = d.str.replace(',', '').astype(int)
            total = d.sum()
            second_try.append(total)
        else:
            second_try.append(0)
    except:
        second_try.append(0)
miss['비정규직 수'] = second_try


C:\Users\pmsda\AppData\Local\Temp\ipykernel_13056\3585036055.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  miss['비정규직 수'] = second_try


In [19]:
df.loc[miss.index] = miss
df

,Unnamed: 0,corp_code,재무제표,부채총계,법인세차감전 순이익,영업이익,정직원 수,비정규직 수
0,0,00106641,"55,344,798,000,000","26,366,660,000,000","2,531,104,000,000","2,009,680,000,000",0.0,0.0
1,1,00138279,"16,333,401,000,000","9,836,775,000,000","110,351,000,000","420,077,000,000",726.0,32.0
2,2,00155355,"1,484,556,453,749","1,021,454,494,037","6,886,489,346","30,570,664,872",412.0,29.0
3,3,00190321,"34,061,292,000,000","18,874,339,000,000","979,676,000,000","1,151,065,000,000",22810.0,562.0
4,4,00181712,"132,020,676,000,000","79,838,041,000,000","2,776,568,000,000","3,949,864,000,000",824.0,130.0
...,...,...,...,...,...,...,...,...
785,785,00161781,"845,966,626,271","452,567,634,439","-3,457,931,698","4,710,346,637",376.0,26.0
786,786,00306162,"3,627,156,734,872","3,008,493,635,568","163,507,886,173","182,373,179,358",168.0,48.0
787,787,00788773,"195,715,540,240","42,889,887,717","27,704,581,004","22,423,158,244",308.0,6.0
788,788,01061558,"181,310,980,033","21,368,228,323","20,781,178,499","20,761,496,950",175.0,6.0


In [35]:
df.to_csv('2019_info.csv',index=False)

In [33]:
df[df['정직원 수']==0]

,Unnamed: 0,corp_code,재무제표,부채총계,법인세차감전 순이익,영업이익,정직원 수,비정규직 수
0,0,00106641,"55,344,798,000,000","26,366,660,000,000","2,531,104,000,000","2,009,680,000,000",0.0,0.0
12,12,00160047,"3,607,086,237,555","212,953,070,700","196,706,021,587","170,875,981,774",0.0,0.0
43,43,00164742,"194,512,220,000,000","118,146,466,000,000","4,163,766,000,000","1,580,164,000,000",0.0,0.0
45,45,00105952,"10,763,512,000,000","6,492,184,000,000","160,438,000,000","351,981,000,000",0.0,0.0
48,48,00140177,"7,054,847,638,519","4,504,006,759,724","199,839,393,215","238,835,955,920",0.0,0.0
82,82,00219097,"1,657,381,460,248","129,126,735,585","26,690,490,888","29,385,396,571",0.0,0.0
83,83,00126308,"4,565,711,799,514","3,270,525,561,644","437,779,765,527","422,717,496,667",0.0,0.0
94,94,00160232,"957,238,168,720","676,238,454,283","23,713,572,228","52,771,937,110",0.0,0.0
95,95,00124540,"9,697,696,817,049","7,209,447,915,772","307,274,974,856","364,061,074,652",0.0,0.0
100,100,00412597,"2,503,844,833,978","774,839,980,801","125,740,946,152","130,344,227,510",0.0,0.0


In [34]:
dart.report('00160047','직원',2019)

,rcept_no,corp_cls,corp_code,corp_name,sexdstn,fo_bbm,reform_bfe_emp_co_rgllbr,reform_bfe_emp_co_cnttk,reform_bfe_emp_co_etc,rgllbr_co,rgllbr_abacpt_labrr_co,cnttk_co,cnttk_abacpt_labrr_co,sm,avrg_cnwk_sdytrn,fyer_salary_totamt,jan_salary_am,rm
0,20221215000222,Y,00160047,한국앤컴퍼니,남,-,-,-,-,48,-,-,-,48,9.4,"4,326,000,000","90,000,000",-
1,20221215000222,Y,00160047,한국앤컴퍼니,여,-,-,-,-,13,-,-,-,13,8.1,"849,000,000","65,000,000",-


In [1]:
import pandas as pd

df = pd.read_csv('2019_info.csv',dtype = {'corp_code':str})
df.head()

,Unnamed: 0,corp_code,재무제표,부채총계,법인세차감전 순이익,영업이익,정직원 수,비정규직 수
0,0,00106641,"55,344,798,000,000","26,366,660,000,000","2,531,104,000,000","2,009,680,000,000",0.0,0.0
1,1,00138279,"16,333,401,000,000","9,836,775,000,000","110,351,000,000","420,077,000,000",726.0,32.0
2,2,00155355,"1,484,556,453,749","1,021,454,494,037","6,886,489,346","30,570,664,872",412.0,29.0
3,3,00190321,"34,061,292,000,000","18,874,339,000,000","979,676,000,000","1,151,065,000,000",22810.0,562.0
4,4,00181712,"132,020,676,000,000","79,838,041,000,000","2,776,568,000,000","3,949,864,000,000",824.0,130.0


In [4]:
corp = df[df['정직원 수']==0]['corp_code']
dart.company(corp[0])['stock_code']

'000270'

In [5]:
st  = []
for code in corp:
    st_code = dart.company(code)['stock_code']
    st.append(st_code)

In [ ]:

url = f"https://comp.kisline.com/co/CO0100M010GE.nice?stockcd={code}&nav=2"
